In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from hashlib import md5

# 1. Хеш-функция

Хеш-функция преобразует хешируемый объект в некоторый набор символов. Этот набор символов называется хешем.

Свойства хеш-функции:

- незначительное изменение входной информации сильно изменяет хеш;
- хеш-функция необратима и не позволяет восстанавливать исходный массив информации из символьной строки;
- быстро вычисляется;
- хеш-функция может приводить любой объем данных к числу заданной длины.

In [2]:
# Хешируем строку
md5('Hello'.encode())

<md5 HASH object @ 0x7f3920d27f70>

In [3]:
# Приводим к 16-ричному числу
md5('Hello'.encode()).hexdigest()

'8b1a9953c4611296a827abf8c47804d7'

In [4]:
# Приводим к 10-тичному числу
int(md5('Hello'.encode()).hexdigest(), 16)

184900800977808474752697256094572479703

In [5]:
# Немного изменим строку, хеш изментся сильно
int(md5('Hell!'.encode()).hexdigest(), 16)

140567244002313857870351343598198988344

С помощью хэш-функции можно «случайно» распределять объекты по группам, если у них есть уникальные идентификаторы.

In [6]:
def get_bucket(value: str, n: int, salt: str=''):
    """Определяет бакет по id.

    value - уникальный идентификатор объекта.
    n - количество бакетов.
    salt - соль для перемешивания.
    """
    hash_value = int(md5((value + salt).encode()).hexdigest(), 16)
    return hash_value % n

In [7]:
n = 100
[get_bucket(str(x), n, 'salt_one') for x in range(10)]

[91, 95, 2, 50, 70, 10, 52, 11, 27, 62]

In [8]:
# если вычислим бакеты для тех же объектов с той же солью, то получим те же значения
[get_bucket(str(x), n, 'salt_one') for x in range(10)]

[91, 95, 2, 50, 70, 10, 52, 11, 27, 62]

In [9]:
# если поменяем солью, то получим другие значения
[get_bucket(str(x), n, 'salt_two') for x in range(10)]

[9, 85, 72, 2, 28, 2, 45, 31, 7, 47]

# 2. Данные пиццерии

На нашей платформе А/Б тестирования реализована очень простая схема формирования групп для эксперимента.

- для эксперимента генерируется случайная соль;
- выбирается количество бакетов;
- пользователи разбиваются по бакетам с солью эксперимента;
- первый бакет — контрольная группа, второй бакет — экспериментальная группа.

Посмотрим, как это работает на примере данных первого эксперимента.

- даты — с 2022-03-23 по 2022-03-30
- количество бакетов — 3
- salt — uSuuwtPc

In [10]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_users = read_database('experiment_users.csv')

df_sales - информация о покупках, одна строка - один заказ. Атрибуты:
- sale_id - идентификатор покупки;
- date - дата покупки;
- count_pizza - количество пицц в заказе;
- count_drink - количество напитков в заказе;
- price - стоимость заказа;
- user_id - идентификатор пользователя;

df_users - список пользователей, попавших в первый эксперимент. Флаг в столбце pilot указывает на группу, 1 - экспериментальная, 0 - контрольная.

In [11]:
df_users.sample(5)

,user_id,pilot
5587,1269da,0
19772,27aba3,1
6111,4985b8,0
3906,dfea33,0
16081,1eac69,1


Посчитаем бакеты сами.

In [12]:
salt = 'uSuuwtPc'
n = 3

df_users['bucket'] = [get_bucket(user_id, n, salt) for user_id in df_users['user_id'].values]

In [13]:
df_users.sample(5)

,user_id,pilot,bucket
14903,5119de,1,1
12820,bfd7e8,1,1
15981,672b2b,1,1
2032,329b33,0,0
1791,d68756,0,0


In [14]:
(df_users['pilot'] == df_users['bucket']).mean()

1.0

Совпадение 100%

Проверим, что для пользователей, которые не попали в эксперимент, номер бакета не 0 и не 1.

In [15]:
all_users = (
    df_sales
    [(df_sales['date'] >= datetime(2022, 3, 23)) & (df_sales['date'] < datetime(2022, 3, 30))]
    ['user_id'].unique()
)

In [16]:
third_bucket_users = list(set(all_users) - set(df_users['user_id'].unique()))

In [17]:
for user_id in third_bucket_users[:10]:
    print(user_id, get_bucket(user_id, n, salt))

6487c2 2
7ec94a 2
9becb5 2
860631 2
423299 2
af85f3 2
8d3ffc 2
fe98a7 2
4550c9 2
f43b8a 2


Всё верно, бакет не 0 и не 1.

# 3. Двойное хеширование

Посмотрим, как можно реализовать двойное хеширование.

Сначала нужно разбить пользователей по бакетам

In [18]:
n_buckets = 100
bucket_salt = 'abc123'

# сделаем датафрейм с пользователями
df = df_sales[['user_id']].drop_duplicates()
# распределим пользователей по бакетам
df['bucket'] = [get_bucket(user_id, n_buckets, bucket_salt) for user_id in df['user_id'].values]

In [19]:
df.head()

,user_id,bucket
0,1c1543,99
1,a9a6e8,83
2,23420a,23
3,3e8ed5,14
4,cbc468,41


Допустим, мы оценили, что для проведения эксперимента нам нужно 5% пользователей. При условии, что у нас 100 бакетов, для эксперимента нужно 5 бакетов. Выберем случайные 5 бакетов и распределим попавших в них пользователей на контрольную и экспериментальную группу.

In [20]:
experiment_buckets = np.random.choice(df['bucket'].unique(), 5, False)
print(f'experiment_buckets: {experiment_buckets}')

df_experiment = df[df['bucket'].isin(experiment_buckets)].copy()
experiment_salt = 'qwe456'
df_experiment['pilot'] = [get_bucket(user_id, 2, experiment_salt) for user_id in df_experiment['user_id'].values]

experiment_buckets: [42 94 31  7 79]


In [21]:
df_experiment.head()

,user_id,bucket,pilot
18,979634,94,1
21,b6a894,42,0
41,cf8d27,42,1
59,5138f1,94,1
98,4951db,79,1


In [22]:
experiment_salt = 'qwe45sadjfb'
df_experiment['pilot2'] = [get_bucket(user_id, 2, experiment_salt) for user_id in df_experiment['user_id'].values]

In [23]:
df_experiment.head(20)

,user_id,bucket,pilot,pilot2
18,979634,94,1,1
21,b6a894,42,0,0
41,cf8d27,42,1,0
59,5138f1,94,1,0
98,4951db,79,1,0
122,24b0fe,79,0,1
147,da64f8,31,1,0
186,48f20a,94,1,0
235,550016,79,1,1
243,437c44,42,0,0
